# Análisis y traducción de tweets de texto común a ASCII

Notebook generadora del dataset para utilizar en el análisis. Toma la base de datos, remueve las palabras y signos a ignorar, traduce los emojis a ASCII y guarda el dataset final.

In [1]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from TweetAnalysis import hermes

import nltk
from nltk.stem import SnowballStemmer

#### Cargo la base de datos clasificada y genero las variables de importancia

In [2]:
TW = pd.read_csv('base_corr.csv')

Estos son los tweets no clasificados

In [3]:
DATA = pd.read_csv('tweets.csv',sep=';')

MSJ contiene al tweet, Tag si es meteorológico o no, Num el id del tweet, Who el nombre de quien los clasificó y User el id de quien twiteó (eso lo tuvimos que sacar de la base de datos original).

In [4]:
MSJ = TW['Twit']
TAG = TW['Tag']
NUM = TW['Num']
WHO = TW['Nombre']
USER = DATA['user_id'][np.asarray(NUM)]

In [5]:
FINAL = pd.DataFrame(list(zip(MSJ, TAG, NUM, USER, WHO)), columns = ['MSJ', 'TAG', 'NUN', 'USER', 'WHO'])

## La función Hermes: normaliza los tweets y borra el spam meteorológico

In [48]:
FINAL = hermes(FINAL, 'MSJ', 'USER', 5)
FINAL.to_csv(r'BaseOrt.csv', sep = ';', index = None)

## Corrector ortográfico

A mano :( Se puede hacer con Hunspell, si lo lográs instalar, suerte!

In [7]:
ORT = pd.read_csv('BaseOrt2.csv', sep = ';', header = None)
ORT.columns = ['msj_ort']

Aplico NLTK

In [42]:
espaniol_stemmer = SnowballStemmer('spanish')
ORT_stemm = []
for i in ORT['msj_ort']:
    msj = i.lower().split()
    for j in range(len(msj)):
        msj[j] = espaniol_stemmer.stem(msj[j])
    i = ' '.join(msj)
    ORT_stemm.append(i)
ORT['msj_stemm'] = ORT_stemm

Otra opción: le saco los plurales

In [44]:
ORT_no_ese = []
for i in ORT['msj_ort']:
    msj = i.lower().split()
    for j in range(len(msj)):
        if (msj[j][-1] == 's') and (msj[j]!='mas'): msj[j]= msj[j][:-1]
    i = ' '.join(msj)
    ORT_no_ese.append(i)
ORT['msj_noplural'] = ORT_no_ese

Remuevo emojis

In [45]:
ORT_no_emoji = []
for i in ORT['msj_ort']:
    msj = i.lower().split()
    for j in range(len(msj)):
        if '\\u' in msj[j]: msj[j] = ' '
    i = ' '.join(msj)
    ORT_no_emoji.append(i)
ORT['msj_noemoji'] = ORT_no_emoji

In [49]:
ORT.to_csv(r'BaseRed.csv', sep = ';', index = None)